# HEALPix Python Project

Author: Yannick Hénin
University of Strasbourg

First we import our python module

In [94]:
from moc_library import Loader, Moc_tree

We define useful constant in this cell such as:
- name of the catalogue to query
- max number of elements in the table
- order of the initial healpix cell

__Note :__ Below are some usable catalogues:

    - 'II/7A/catalog' : UBVRIJKLMNH Photoelectric Catalogue (Morel+ 1978)
    - 'J/ApJ/831/67/table1' : Galactic CHaMP. III. 12CO dense clump properties (Barnes+, 2016)
    - 'J/ApJ/804/L15/data' : SDSS-DR7 broad-line QSOs (Sun+, 2015)
    - 'VII/26D/catalog' : Uppsala General Catalogue of Galaxies (UGC) (1973)
    - 'I/298/table3' :  Catalog of Northern stars with annual proper motions larger than 0.15" (2005)
    - 'V/148/morx'
    - 'J/ApJS/236/30/hatlas2ngp'

In [95]:
CATALOGUE = 'J/ApJ/804/L15/data'
OUT_MAX: int|str = "unlimited" # One can specify a limit number or 'unlmited'
ORDER = 8

The library is built using the astropy library, it is hence able to query the table from Vizier easily. 
This step is made by the Loader class which has only two methods:
- get_votable() : Allow one to get the votable
- build_healpix_table() : Construct a healpix column from (ra,dec) and add it to the table

Let's first query the catalogue and have a look at the table queried.
__Note :__ The query can be made from CDS (Strasbourg) Vizier or CFA (Harvard) Vizier, this is specified in the source argument 

In [96]:
loader = Loader(_catalogue=CATALOGUE, _out_max=OUT_MAX, _order=ORDER, _source='CDS')
votable = loader.get_votable()
print(votable)

Querying: 'J/ApJ/804/L15/data' from Vizier
Coordinate system of the table: eq_FK5

 MJD  Plate Fiber  RAJ2000  ... DR7_DR9  Sp        SimbadName        NED
                     deg    ...                                         
----- ----- ----- --------- ... ------- --- ------------------------ ---
51602   266   254 145.46519 ... DR7+DR9  Sp SDSS J094151.64-010352.0 NED
51630   266   200 145.92530 ... DR7+DR9  Sp SDSS J094342.06+000255.7 NED
51630   266   227 145.62226 ... DR7+DR9  Sp SDSS J094229.34+000204.6 NED
51630   266   494 146.06576 ... DR7+DR9  Sp SDSS J094415.77+001101.2 NED
51630   266   557 146.81401 ... DR7+DR9  Sp SDSS J094715.36+000708.1 NED
51608   267    78 149.10806 ... DR7+DR9  Sp SDSS J095625.93-000353.0 NED
51608   267   372 147.57777 ... DR7+DR9  Sp SDSS J095018.66+005049.8 NED
51608   267   593 148.88333 ... DR7+DR9  Sp SDSS J095532.00+004108.7 NED
51910   269   147 151.37488 ... DR7+DR9  Sp SDSS J100529.98-004152.7 NED
51910   269   281 149.99127 ... DR7+DR9  

Let's now add the HEALPix column to this VOTable. <br>
**Note:** The _build_healpix_table()_ method sorts the table and removes the duplicated lines in the healpix table automatically to avoid overlapping problems in the future. 
To avoid this, one can specify whether to do it or not with the argument *_rm_duplicates*.


In [97]:
healpix_table = loader.build_healpix_table(votable)
print(healpix_table)

Removed 53 duplicates.

 MJD  Plate Fiber  RAJ2000  ...  Sp        SimbadName        NED HEALPix
                     deg    ...                                         
----- ----- ----- --------- ... --- ------------------------ --- -------
52556   808   377  45.24006 ...  Sp SDSS J030057.61+004147.9 NED      30
52205   709   430  44.90895 ...  Sp SDSS J025938.16+004216.3 NED      50
52205   709   411  45.20391 ...  Sp SDSS J030048.94+005440.7 NED      53
52289   802   345  44.96145 ...  Sp SDSS J025950.74+010304.6 NED      62
51817   411   403  45.35109 ...  Sp SDSS J030124.26+011022.8 NED      99
51817   411   413  45.43417 ...  Sp SDSS J030144.19+011530.8 NED     102
52175   708   578  44.01069 ...  Sp SDSS J025602.56+005255.3 NED     138
52175   708   564  44.19568 ...  Sp SDSS J025646.96+011349.2 NED     165
53733  2340   221  47.99845 ...  Sp SDSS J031159.63+041803.4 NED    1243
51883   428   108  33.32624 ...  Sp SDSS J021318.30+130643.8 NED   33682
  ...   ...   ...       ...

In [98]:
print(loader.build_healpix_table(votable[:5], _rm_duplicates=False))
print('______________________________________________________________________________________________________________________')
print(healpix_table[:3])

Duplicates were not removed.

 MJD  Plate Fiber  RAJ2000  ...  Sp        SimbadName        NED HEALPix
                     deg    ...                                         
----- ----- ----- --------- ... --- ------------------------ --- -------
51602   266   254 145.46519 ...  Sp SDSS J094151.64-010352.0 NED 1740722
51630   266   200 145.92530 ...  Sp SDSS J094342.06+000255.7 NED 1746268
51630   266   227 145.62226 ...  Sp SDSS J094229.34+000204.6 NED 1746266
51630   266   494 146.06576 ...  Sp SDSS J094415.77+001101.2 NED 1749000
51630   266   557 146.81401 ...  Sp SDSS J094715.36+000708.1 NED 1743538
______________________________________________________________________________________________________________________
 MJD  Plate Fiber  RAJ2000  ...  Sp        SimbadName        NED HEALPix
                     deg    ...                                         
----- ----- ----- --------- ... --- ------------------------ --- -------
52556   808   377  45.24006 ...  Sp SDSS J030057

Some lines seem indeed to be duplicated in the new table.

Now that we have our Table well-prepared, one can continue by creating a MOC tree. This is done by the Moc_tree class. <br>

This class has 4 methods:
- _build_moc_tree()_ : create the tree with from a HEALPix table
- _serialize_moc()_ : return the serialized moc tree
- _add_node()_ : add a node to a given order in the tree
- _del_nodes()_ : remove different nodes from a given order in the tree

Let's now build our Moc_tree.

In [99]:
moc_tree = Moc_tree(ORDER)
print(moc_tree.nodes)

{9: []}


There is nothing but the initial order in the tree nodes yet.<br> <br>

Be careful to the table that you give to the moc_tree object. It needs to contain a HEALPix column. Otherwise, it will automatically raise an error.

In [100]:
# Automatically raises an error if the wrong table is given
# moc_tree.build_moc_tree(votable)

In [101]:
moc_tree.build_moc_tree(healpix_table)
print(moc_tree.nodes)

{9: [30, 50, 53, 62, 99, 102, 138, 165, 1243, 33682, 33694, 34995, 35067, 35140, 57893, 109043, 133813, 142659, 169037, 171510, 171511, 172047, 172068, 181166, 182436, 262165, 262204, 262228, 262295, 262340, 262368, 262412, 262442, 262610, 262669, 262755, 262899, 263052, 263080, 263224, 263290, 263394, 263435, 263557, 263574, 263669, 263773, 263842, 264087, 264230, 264324, 264498, 264697, 264739, 264785, 264811, 264842, 264962, 265005, 265259, 265346, 265388, 265409, 265445, 265598, 265638, 265755, 265787, 265853, 265903, 265963, 265980, 266010, 266043, 266252, 266339, 266420, 266504, 266629, 266795, 266826, 266862, 266870, 266914, 266930, 266961, 267001, 267094, 267104, 267290, 267297, 267320, 267355, 267394, 267436, 267534, 267585, 267610, 267657, 267659, 267669, 267716, 267758, 267827, 267861, 267868, 267874, 267985, 268014, 268024, 268052, 268089, 268096, 268110, 268121, 268140, 268243, 268266, 268275, 268312, 268581, 268677, 268881, 268939, 269111, 269117, 269146, 269152, 269236, 

The tree is built. Let's have a look at the serialization of the tree

In [102]:
print(moc_tree.serialize_moc())

9/30 50 53 62 99 102 138 165 1243 33682 33694 34995 35067 35140 57893 109043 133813 142659 169037 171510 171511 172047 172068 181166 182436 262165 262204 262228 262295 262340 262368 262412 262442 262610 262669 262755 262899 263052 263080 263224 263290 263394 263435 263557 263574 263669 263773 263842 264087 264230 264324 264498 264697 264739 264785 264811 264842 264962 265005 265259 265346 265388 265409 265445 265598 265638 265755 265787 265853 265903 265963 265980 266010 266043 266252 266339 266420 266504 266629 266795 266826 266862 266870 266914 266930 266961 267001 267094 267104 267290 267297 267320 267355 267394 267436 267534 267585 267610 267657 267659 267669 267716 267758 267827 267861 267868 267874 267985 268014 268024 268052 268089 268096 268110 268121 268140 268243 268266 268275 268312 268581 268677 268881 268939 269111 269117 269146 269152 269236 269285 269325 269356 269434 269541 269658 269720 269769 269803 269811 269827 270100 270137 270145 270194 270265 270308 270387 270512

### Comparison of the results
Now that we have serialized our moc tree, let's compare the results of our custom MOC library with the official mocpy library

In [103]:
from mocpy import MOC

# A MOC object can be obtained from our serialization using MOC.from_string()
custom_moc = MOC.from_string(moc_tree.serialize_moc())

print(custom_moc)

9/30 50 53 62 99 102 138 165 1243 33682 33694 34995 35067 35140 57893 109043 
 133813 142659 169037 171510-171511 172047 172068 181166 182436 262165 262204 
 262228 262295 262340 262368 262412 262442 262610 262669 262755 262899 263052 
 263080 263224 263290 263394 263435 263557 263574 263669 263773 263842 264087 
 264230 264324 264498 264697 264739 264785 264811 264842 264962 265005 265259 
 265346 265388 265409 265445 265598 265638 265755 265787 265853 265903 265963 
 265980 266010 266043 266252 266339 266420 266504 266629 266795 266826 266862 
 266870 266914 266930 266961 267001 267094 267104 267290 267297 267320 267355 
 267394 267436 267534 267585 267610 267657 267659 267669 267716 267758 267827 
 267861 267868 267874 267985 268014 268024 268052 268089 268096 268110 268121 
 268140 268243 268266 268275 268312 268581 268677 268881 268939 269111 269117 
 269146 269152 269236 269285 269325 269356 269434 269541 269658 269720 269769 
 269803 269811 269827 270100 270137 270145 270194 270

In [104]:
# One can query the table using the mocpy library as well
moc = MOC.from_vizier_table(table_id=CATALOGUE, nside=2**ORDER)
print(moc)

9/30 50 53 62 99 102 138 165 1243 33682 33694 34995 35067 35140 57893 109043 
 133813 142659 169037 171510-171511 172047 172068 181166 182436 262165 262204 
 262228 262295 262340 262368 262412 262442 262610 262669 262755 262899 263052 
 263080 263224 263290 263394 263435 263557 263574 263669 263773 263842 264087 
 264230 264324 264498 264697 264739 264785 264811 264842 264962 265005 265259 
 265346 265388 265409 265445 265598 265638 265755 265787 265853 265903 265963 
 265980 266010 266043 266252 266339 266420 266504 266629 266795 266826 266862 
 266870 266914 266930 266961 267001 267094 267104 267290 267297 267320 267355 
 267394 267436 267534 267585 267610 267657 267659 267669 267716 267758 267827 
 267861 267868 267874 267985 268014 268024 268052 268089 268096 268110 268121 
 268140 268243 268266 268275 268312 268581 268677 268881 268939 269111 269117 
 269146 269152 269236 269285 269325 269356 269434 269541 269658 269720 269769 
 269803 269811 269827 270100 270137 270145 270194 270

Let's check if there are any difference between my serialization and the one that comes from MOCpy. 
I am defining the following function that takes in argument two strings and returns the different elements between the two strings

In [105]:
def difference(string1: str, string2: str):
    # Split both strings into list items
    string1 = string1.split()
    string2 = string2.split()

    set1 = set(string1)
    set2 = set(string2)

    str_diff = set1.symmetric_difference(set2)

    if len(str_diff) == 0:
        print("The MOC strings are the same")
    else:
        print(f"Difference between two MOC strings:\n{str_diff}")


difference(custom_moc.to_string(), moc.to_string())

The MOC strings are the same


Nice, now that we have everything, seeing how they appear on a map would be interesting to verify our work.
<end>

In [106]:
import matplotlib.pyplot as plt
%matplotlib notebook

# Create a figure and axis
fig = plt.figure(figsize=(10, 10))
wcs = custom_moc.wcs(fig)
ax = fig.add_subplot(projection=wcs)

# # Plot the custom MOC in red and the standard MOC in blue
custom_moc.fill(ax, wcs, color='blue', alpha=0.5, label='Custom MOC')
moc.fill(ax, wcs, color='red', alpha=0.5, label='MOCpy')

plt.title("Projection of the HEALPix cells")
plt.legend()
plt.show()  

<IPython.core.display.Javascript object>

/home/yhenin/miniconda3/envs/healpix/lib/python3.11/site-packages/matplotlib/backends/backend_nbagg.py:181: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm('matplotlib', data={'id': self.uuid})


## ipyaladin
Now that we have our moc object, one can try to use it in Aladin

The HEALPix seem to overlap correctly. That attests that the library seems to work just fine.
<br>
__Note:__ For some catalogues, one can see some outliers. I think that this comes from the MOC library that misses some data.

In [116]:
from ipyaladin import Aladin

custom_moc_json = custom_moc.serialize(format="json")

# Display the MOC using ipyaladin
aladin = Aladin()
aladin.add_moc_from_dict(custom_moc_json, {'color': 'red', 'opacity': 0.3})
aladin

Aladin(moc_dict={'9': [30, 50, 53, 62, 99, 102, 138, 165, 1243, 33682, 33694, 34995, 35067, 35140, 57893, 1090…